In [0]:
%sql
USE CATALOG bikestore;

In [0]:
# Definir pastas do projetos em variaveis para facilitar
bronze_path   = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/bronze/'
silver_path   = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/silver/'
gold_path     = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/gold/'
resource_path = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/resource/origem'
resource_path_volume = '/Volumes/bikestore/logistics/bikestore_rescource/origem/'

In [0]:
display(dbutils.fs.ls(f'{bronze_path}'))

In [0]:
display(dbutils.fs.ls(f'{bronze_path}/brands/'))

In [0]:
(spark.read.format('delta')
  .load(f'{bronze_path}/customers/')
  .createOrReplaceTempView('tmp_customers')
)


In [0]:
%sql
select * from tmp_customers;

In [0]:
%sql
DESCRIBE tmp_customers;

In [0]:
display(dbutils.fs.ls("abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/bronze/"))

In [0]:
# criar varias tabelas temporárias de forma prática
bronze_map = {
    "tmp_bronze_brands":      f"{bronze_path}/brands/",
    "tmp_bronze_categories":  f"{bronze_path}/categories/",
    "tmp_bronze_customers":   f"{bronze_path}/customers/",
    "tmp_bronze_order_items": f"{bronze_path}/order_details/",
    "tmp_bronze_orders":      f"{bronze_path}/orders/",
    "tmp_bronze_products":    f"{bronze_path}/products/",
    "tmp_bronze_staffs":      f"{bronze_path}/staffs/",
    "tmp_bronze_stocks":      f"{bronze_path}/stocks/",
    "tmp_bronze_stores":      f"{bronze_path}/stores/",
}
for view_name, path in bronze_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))
 


In [0]:
%sql
select * from tmp_bronze_brands;

In [0]:
%sql
show tables in bikestore.logistics;

#######CAMDADA SILVER

In [0]:
%sql
select * from tmp_bronze_categories;

In [0]:
%sql
select * from tmp_bronze_products;

In [0]:
%sql
DESCRIBE tmp_bronze_products;

In [0]:
%sql
select
  p.product_id,
  p.product_name,
  p.brand_id,
  c.category_name,
  p.model_year,
  p.list_price
from tmp_bronze_products as p
  left join tmp_bronze_categories as c 
  on c.category_id = p.category_id;

In [0]:
df_product_silver = spark.sql("""
    select
        p.product_id,
        p.product_name,
        p.brand_id,
        c.category_name,
        p.model_year,
        p.list_price
    from tmp_bronze_products as p
        left join tmp_bronze_categories as c 
        on c.category_id = p.category_id;
                              
                              """)

In [0]:
display(df_product_silver)

In [0]:
df_product_silver.write\
                 .mode("overwrite") \
                 .format("delta") \
                 .option("mergeSchema", "true") \
                 .save(f"{silver_path}/product_teste")

In [0]:
display( dbutils.fs.ls(silver_path) )

In [0]:
%sql
CREATE TABLE IF NOT EXISTS bikestore.logistics.products_teste
LOCATION "abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/silver/product_teste/";